In [94]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import serial

import glob
import msvcrt
import winsound
import sklearn
import re
import csv

import time
import os
import shutil
import sys
from datetime import datetime
print(sys.version)
print(sys.executable)

3.5.6 |Anaconda, Inc.| (default, Aug 26 2018, 16:05:27) [MSC v.1900 64 bit (AMD64)]
C:\Anaconda3\envs\nectarCamera\python.exe


In [95]:
# define directories
baseDir = os.getcwd()

# may want to make this directory somewhere else, if dropbox becomes a problem
dataDir = r"D:\Dropbox\AcademiaDropbox\UW\BeeDecisionProject\NectarData_servo"
if not os.path.isdir(dataDir):
    os.mkdir(dataDir)


figDir = r"D:\Dropbox\AcademiaDropbox\UW\BeeDecisionProject\NectarFigs_servo"
if not os.path.isdir(figDir):
    os.mkdir(figDir)


In [96]:
int.from_bytes(b'u', byteorder='big')

117

In [97]:
# list ports

def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result


# I'm using COM3 and COM5 arduinos
# COM4 is for the arduino controlling the cameras
serial_ports()

['COM1', 'COM4']

In [5]:
# connect 
PORT1 = "COM5"
connected1 = False
if "ser1" in globals():
    ser1.close()
ser1 = serial.Serial(PORT1,19200, timeout=1.0) # stop if no data comes in 1 second
while not connected1:
    serin1 = ser1.read()
    connected1 = True
    print("connected to arduino on " + PORT1)
str(ser1.readline().decode("UTF-8"))    

connected to arduino on COM5


''

In [6]:
ser1.write("d".encode("utf-8"))


1

In [7]:
ser1.write("u".encode("utf-8"))

1

In [58]:
def get_photogate_data(ser1):
    ser1.write("r".encode("utf-8"))
    return(int(ser1.readline().decode("utf-8")))

In [59]:
%%timeit
[get_photogate_data(ser1) for ii in range(100)]

525 ms ± 25.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
# basic structure


# read in photosensor data
# get a good idea of baseline....
# if the baseline changes, trigger reward for 2 seconds, then go back down

def get_baseline(ser1):
    bl=  [get_photogate_data(ser1) for ii in range(1000)]
    # return upper and lower bounds
    np.mean(bl), np.std(bl)
    return(((np.mean(dta) - 3* np.std(dta)),(np.mean(dta) + 3* np.std(dta)))) 

photogate_baseline = get_baseline(ser1)






In [104]:
# start with nectar down
ser1.write("d".encode("utf-8"))
servoPosition = "down"
upTime = datetime.now()
rewardTime = 2 # seconds
photogateMinValue = 9999
readyToReward = True


for ctr in range(10000):
    
    
    while msvcrt.kbhit():
        msvcrt.getch()
        print('clearing characters ...')
    
    if ctr == 0:
        saveFileName = (datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f'))
        saveFileName = re.sub(r'[^\w\s]','_',saveFileName)
        saveFileName = re.sub(" ", "__", saveFileName)[0:] + ".csv"
        
        # write header
        with open(os.path.join(dataDir, saveFileName), 'w+', newline='') as myfile:
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerows([["datetime", "photogate_value", "nectar_position"]])       
    
    
    # get data
    photogate_value = get_photgate_data(ser1)
    photogateMinValue = np.min([photogateMinValue, photogate_value])

    
    # process data
    # if beam is broken, bring up nectar and start timer
    if ((photogate_value > photogate_baseline[1]) & 
        (servoPosition == "down") & 
        (readyToReward)):
        servoPosition = "up"
        ser1.write("u".encode("utf-8"))
        upTime = datetime.now()
        photogateMinValue = 9999
        readyToReward = False
        
    if ((datetime.now() - upTime).total_seconds() > rewardTime) :
        ser1.write("d".encode("utf-8"))
        servoPosition = "down"
    
    if photogateMinValue < photogate_baseline[1]:
        readyToReward = True
#     print((datetime.now() - upTime).total_seconds())
    # if insect doesn't remove its tongue within 2 sec, it should go down
    
    # stream data to file
    # timestamp, beam value, nectar_access
    tmp = [(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')), photogate_value, servoPosition ]
    with open(os.path.join(dataDir, saveFileName), 'a+', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerows([tmp])
    
    # break by keyboard option
    if msvcrt.kbhit(): # if q, or escape is pressed, then break
        k = msvcrt.getch()
        if(k == b'q') | (k == b'\x1b') | (k == b'\x0b') :
            print("keyboard break")
            winsound.MessageBeep()
            break
    

keyboard break


In [101]:
tmp

['2019-08-29 15:02:02.260775', 768, 'down']

In [85]:
tmp = [(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')), photogate_value, servoPosition ]
tmp

['2019-08-29 14:53:26.614940', 768, 'down']

In [46]:
(datetime.now() - upTime).total_seconds()

44.37655

In [89]:
saveFileName = (datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f'))
saveFileName = re.sub(r'[^\w\s]','_',saveFileName)
saveFileName = re.sub(" ", "__", saveFileName)[0:] + ".csv"

saveFileName

'2019_08_29__14_56_29_351527.csv'

In [40]:
get_baseline(ser1)

(766.2592677056812, 769.0867322943188)

In [34]:
dta = [get_photgate_data(ser1) for ii in range(1000)]
#dta = np.random.normal(size = 1000)

In [35]:
np.mean(dta), np.std(dta)

(767.673, 0.47124409810627865)

In [31]:
# this would be a 95% CI for a normal dist
np.mean(((np.mean(dta) - 2* np.std(dta)) < dta) & (dta < (np.mean(dta) + 2* np.std(dta))) )

0.95

In [ ]:
for ii in range(100): 
    written = ser1.write("b".encode("utf-8"))

In [ ]:
"c".encode("utf-8")
int.from_bytes(b'c', byteorder='big') # this is what the arduino will see

In [ ]:
int.from_bytes(b'c', byteorder='big')

In [ ]:
ser1.write("c".encode("utf-8"))

In [ ]:
def readData(serial_con, readlen = 500, wait_time = 0):
    tmp = []
    while msvcrt.kbhit():
        msvcrt.getch()
        print('clearing characters ...')
    
    for ii in range(readlen):
        serial_con.write("r".encode("utf-8"))
        txt = serial_con.readline().decode("UTF-8")
        values = [int(i) for i in txt.split(',')]
        values.append(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f'))
        tmp.append(np.array(values))
        time.sleep(wait_time)
        
        if msvcrt.kbhit(): # if q is pressed, then break
            k = msvcrt.getch()
            if(k == b'q') | (k == b'\x1b') | (k == b'\x0b') :
                print("keyboard break")
                winsound.MessageBeep()
                break
    return(np.array(tmp))
    

In [ ]:
# read data and write to file with timestamp
tt = readData(ser1, readlen=1000, wait_time=0.01)

In [ ]:
# saveData
saveDir = r"C:/Users/Combes4/Desktop/NectarPrelim2"

df1 = pd.DataFrame(tt, columns=["base_sensor", "mid_sensor", "top_sensor", "limit_1", "limit_2", "timestamp"])
#df1.to_csv(os.path.join(saveDir, datetime.now().strftime('%Y-%m-%d__%H_%M_%S_%f') + ".csv"), index = False)

In [ ]:

# for com7
df1[["base_sensor", "mid_sensor", 
     "top_sensor", "limit_1", "limit_2"]] = \
    df1[["base_sensor", "mid_sensor", 
        "top_sensor", "limit_1", "limit_2"]].astype(int)

df1.head()

In [ ]:
# for com8, switch base and mid refref, double check
df1[["mid_sensor", "base_sensor",
     "top_sensor", "limit_1", "limit_2"]] = \
    df1[["base_sensor", "mid_sensor", 
        "top_sensor", "limit_1", "limit_2"]].astype(int)

df1.head()

In [ ]:
#np.array(df1.iloc[:,0].astype(int))

In [ ]:
ax1 = df1.iloc[:, 0:3].plot()
lines, labels = ax1.get_legend_handles_labels()
ax1.legend(lines, labels, loc='best')  

In [ ]:
plt.plot(df1[["base_sensor", "mid_sensor", 
     "top_sensor"]])
plt.show()

In [ ]:
plt.plot(np.array(df1.iloc[:,1].astype(int)))

In [ ]:
plt.plot(np.array(df1.iloc[:,2].astype(int)))

In [ ]:
(tt[0, 5])

In [ ]:
# for com7
# top sensor
tt[:, 2]

# mid sensor
tt[:, 1]

# base sensor
tt[:, 0]

In [ ]:
plt.plot(tt[:,2])

In [ ]:
plt.plot(tt)

In [ ]:
#ser1.close()

In [ ]:
def moveToTop(serial_con, cutoff = 650):
    # refref: may want to go 1 or two more moves forward after cutoff is passed --- 
    ## the cutoff is the meniscus
    [[topVal, bottomLim, topLim]] = readData(serial_con, 1, 0)[:, [1,3,4]]
    print(topVal)
    while (topVal > cutoff) and not topLim:
        #move forward
        ser1.write("f".encode("utf-8"))
        
        # read data again
        [[topVal, bottomLim, topLim]] = readData(serial_con, 1, 0)[:,[1,3,4]]
        # rror if limit switch is hit
        if topLim:
            raise RuntimeError('Hit upper limit switch')
        
        
        
        

In [ ]:
# refref: problem -- liquid stays stuck on the sides -- may need to move back slower

def moveBack(serial_con, cutoff = 650):
    
    [[topVal, bottomLim, topLim]] = readData(serial_con, 1, 0)[:, [1,3,4]]
    
    
    while (topVal < cutoff) and not bottomLim:
        #move backward
        for jj in range(7):
            ser1.write("b".encode("utf-8"))
            time.sleep(0.3)
        
        # read data again
        [[topVal, bottomLim, topLim]] = readData(serial_con, 1, 0)[:, [1,3,4]]
        # rror if limit switch is hit
        if bottomLim:
            raise RuntimeError('Hit lower limit switch')

In [ ]:
readData(ser1, 1, 0)

In [ ]:
moveToTop(ser1)
readData(ser1, 1, 0)

In [ ]:
moveBack(ser1)
readData(ser1, 1, 0)

In [ ]:
ser1.write("f".encode("utf-8"))

In [ ]:
[[vals, bottomLim, topLim]] = readData(ser1, 1, 0)[:, 2:]

In [ ]:
readData(ser1, 1, 0)[:, 2:]

In [ ]:
toplim = 1

In [ ]:
not toplim